####  Formulate and solve an optimization problem to reduce the overall power consumption of the machines running together, and save energy

In [9]:
import joblib
import numpy as np
from scipy.optimize import linprog

# Load the KNN models using joblib
model_path_machine1 = '../models/knn_machine1_model.pkl'
model_path_machine2 = '../models/knn_machine2_model.pkl'

loaded_model_machine1 = joblib.load(model_path_machine1)
loaded_model_machine2 = joblib.load(model_path_machine2)

# Define the limits for each machine type
limits_machine1 = (180, 600)  # Machine Type #1
limits_machine2 = (300, 1000)  # Machine Type #2

# Number of machines
num_machines_type1 = 10
num_machines_type2 = 10

# Create the bounds for GPH values for all machines
bounds = [(limits_machine1[0], limits_machine1[1])] * num_machines_type1 + \
         [(limits_machine2[0], limits_machine2[1])] * num_machines_type2

# Target GPH
target_gph = 9000

# Define the power model using loaded KNN models
def power_model(gph_values):
    gph_machine1 = gph_values[:num_machines_type1]  # First 10 for machine type #1
    gph_machine2 = gph_values[num_machines_type1:]   # Last 10 for machine type #2
    
    # Predict power usage for Machine Type #1
    power_machine1 = loaded_model_machine1.predict(np.column_stack((np.full(num_machines_type1, 25), 
                                                                     np.full(num_machines_type1, 6), 
                                                                     gph_machine1)))
    # Predict power usage for Machine Type #2
    power_machine2 = loaded_model_machine2.predict(np.column_stack((np.full(num_machines_type2, 25), 
                                                                     np.full(num_machines_type2, 6), 
                                                                     gph_machine2)))

    return np.concatenate((power_machine1, power_machine2))

# Objective function for optimization
def objective_function(gph_values):
    return sum(power_model(gph_values))

# Linear programming to minimize total power consumption
result = linprog(
    c=[1] * (num_machines_type1 + num_machines_type2),  # Coefficients for GPH values
    A_eq=[np.ones(num_machines_type1 + num_machines_type2)],  # Equality constraint for total GPH
    b_eq=[target_gph],  # Target GPH
    bounds=bounds,
    method='highs'
)

# Retrieve optimal GPH values
optimal_gph_values = result.x
total_power = objective_function(optimal_gph_values)

# Print the results
print("Optimal GPH values for each machine:")
for i in range(num_machines_type1):
    print(f"Machine Type #1 - Machine {i + 1}: {optimal_gph_values[i]:.2f} GPH")
for i in range(num_machines_type2):
    print(f"Machine Type #2 - Machine {i + 1}: {optimal_gph_values[num_machines_type1 + i]:.2f} GPH")
print(f"Total Power Consumption: {total_power:.2f}")


Optimal GPH values for each machine:
Machine Type #1 - Machine 1: 180.00 GPH
Machine Type #1 - Machine 2: 180.00 GPH
Machine Type #1 - Machine 3: 180.00 GPH
Machine Type #1 - Machine 4: 180.00 GPH
Machine Type #1 - Machine 5: 180.00 GPH
Machine Type #1 - Machine 6: 180.00 GPH
Machine Type #1 - Machine 7: 180.00 GPH
Machine Type #1 - Machine 8: 180.00 GPH
Machine Type #1 - Machine 9: 180.00 GPH
Machine Type #1 - Machine 10: 180.00 GPH
Machine Type #2 - Machine 1: 300.00 GPH
Machine Type #2 - Machine 2: 300.00 GPH
Machine Type #2 - Machine 3: 300.00 GPH
Machine Type #2 - Machine 4: 300.00 GPH
Machine Type #2 - Machine 5: 1000.00 GPH
Machine Type #2 - Machine 6: 1000.00 GPH
Machine Type #2 - Machine 7: 1000.00 GPH
Machine Type #2 - Machine 8: 1000.00 GPH
Machine Type #2 - Machine 9: 1000.00 GPH
Machine Type #2 - Machine 10: 1000.00 GPH
Total Power Consumption: 2037.43


/Users/shivampratapsingh/Documents/python_code_dir/assignment/.venv/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
/Users/shivampratapsingh/Documents/python_code_dir/assignment/.venv/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(


#### Output Excel file (`machine_power_data.xlsx`) with the calculated power data for each machine.

In [10]:
import pandas as pd

# Define static inputs
input1 = 25
input2 = 6

# Define optimal GPH values and corresponding power for Machine Type #1
gph_machine1 = [180.00] * 10  # All instances of Machine Type #1
power_machine1 = []  # Placeholder for power values

# Calculate power for Machine Type #1 based on the trained model
for gph in gph_machine1:
    model_path_machine1 = '../models/knn_machine1_model.pkl'
    loaded_model_machine1 = joblib.load(model_path_machine1)
    power = loaded_model_machine1.predict([[input1, input2, gph]])[0]
    power_machine1.append(power)

# Define optimal GPH values and corresponding power for Machine Type #2
gph_machine2 = [300.00] * 4 + [1000.00] * 6  # Four at 300 and six at 1000
power_machine2 = []  # Placeholder for power values

# Calculate power for Machine Type #2 based on the trained model
for gph in gph_machine2:
    model_path_machine2 = '../models/knn_machine2_model.pkl'
    loaded_model_machine2 = joblib.load(model_path_machine2)
    power = loaded_model_machine2.predict([[input1, input2, gph]])[0]
    power_machine2.append(power)

# Create DataFrame for Machine Type #1
df_machine1 = pd.DataFrame({
    'Machine Type': 'Machine Type #1',
    'Input 1': input1,
    'Input 2': input2,
    'GPH': gph_machine1,
    'Power': power_machine1
})

# Create DataFrame for Machine Type #2
df_machine2 = pd.DataFrame({
    'Machine Type': 'Machine Type #2',
    'Input 1': input1,
    'Input 2': input2,
    'GPH': gph_machine2,
    'Power': power_machine2
})

# Combine the two DataFrames
df_combined = pd.concat([df_machine1, df_machine2], ignore_index=True)

# Save to Excel
output_file_path = '../output/machine_power_data.xlsx'
df_combined.to_excel(output_file_path, index=False)

print(f"Data saved to {output_file_path}")


Data saved to ../output/machine_power_data.xlsx


/Users/shivampratapsingh/Documents/python_code_dir/assignment/.venv/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
/Users/shivampratapsingh/Documents/python_code_dir/assignment/.venv/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
/Users/shivampratapsingh/Documents/python_code_dir/assignment/.venv/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
/Users/shivampratapsingh/Documents/python_code_dir/assignment/.venv/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
/Users/shivampratapsingh/Documents/python_co